<a href="https://colab.research.google.com/github/jcmm518/03MAIR-Algoritmos-de-optimizacion/blob/main/Algoritmos_AG2_Juan_Carlos_Marin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos - Actividad Guiada 2

Nombre : Juan Carlos Marin Mejia <br><br>
URL: [https://colab.research.google.com/drive/13n86aVcqwJ5gMTvFYXxqcg2DNZSWYr1X?usp=sharing](https://colab.research.google.com/drive/13n86aVcqwJ5gMTvFYXxqcg2DNZSWYr1X?usp=sharing)

GitHub: [https://github.com/jcmm518/03MAIR-Algoritmos-de-optimizacion.git](https://github.com/jcmm518/03MAIR-Algoritmos-de-optimizacion.git)